In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Wed Apr  3 04:39:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              26W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/

In [3]:
!pip install transformers==4.12.2
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!git clone https://github.com/aub-mind/arabert
!pip install emoji==1.6.1
!pip install sentencepiece==0.1.96
!pip install seqeval
!pip install accelerate

  Using cached transformers-4.12.2-py3-none-any.whl (3.1 MB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
fatal: destination path 'arabert' already exists and is not an empty directory.


In [4]:
!git clone https://github.com/AhmedAbdel-Aal/WNER_24_sharedtask.git
!cp -r WNER_24_sharedtask/code/*.py .
!rm -rf WNER_24_sharedtask

Cloning into 'WNER_24_sharedtask'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 51 (delta 18), reused 48 (delta 15), pack-reused 0
Receiving objects: 100% (51/51), 10.45 KiB | 10.45 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [5]:
import numpy as np
import torch
import random
import copy
import pandas as pd


from arabert.preprocess import ArabertPreprocessor
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModel, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures

from arabert.preprocess import ArabertPreprocessor
import numpy as np

from transformers import AutoConfig, AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
from utils import *

In [6]:
train_data = load_json('/content/drive/MyDrive/anlp24/split70.json')
dev_data = load_json('/content/drive/MyDrive/anlp24/split10.json')

def normalize_data(items):
    normalized_data = []
    text_lists = []
    label_list = []
    for item in items:
        sentence_id = item['global_sentence_id']
        sentence = []
        labels = []
        for token_info in item['tokens']:
            word = token_info['token']
            tv = token_info['tags'][0]
            label = tv['value']
            sentence.append(word)
            labels.append(label)
        #sentence = ' '.join(sentence)
        #labels = ' '.join(labels)
        text_lists.append(sentence)
        label_list.append(labels)
    return text_lists, label_list


# Normalize the data
text_train, labels_train = normalize_data(train_data)
text_dev, labels_dev = normalize_data(dev_data)

data loaded from path /content/drive/MyDrive/anlp24/split70.json
data loaded from path /content/drive/MyDrive/anlp24/split10.json


In [7]:
all_labels = list(set([label for sublist in labels_train for label in sublist]))
label_map = { v:index for index, v in enumerate(all_labels)}
inv_label_map = {i: label for i, label in enumerate(all_labels)}

In [24]:
from data import NERDataset

In [25]:
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'tokenclassification'

In [26]:
tokenizer_path = '/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [27]:
train_dataset = NERDataset(
    texts=text_train,
    tags=labels_train,
    label_list=all_labels,
    model_name=model_name,
    max_length=512,
    tokenizer=tokenizer
    )

dev_dataset = NERDataset(
    texts=text_dev,
    tags=labels_dev,
    label_list=all_labels,
    model_name=model_name,
    max_length=512,
    tokenizer=tokenizer
    )

In [11]:
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [28]:
def model_init_trained():
    return AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir").to(device)

In [29]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)

    batch_size, seq_len = preds.shape

    out_label_list = [[] for _ in range(batch_size)]
    preds_list = [[] for _ in range(batch_size)]

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                out_label_list[i].append(inv_label_map[label_ids[i][j]])
                preds_list[i].append(inv_label_map[preds[i][j]])

    return preds_list, out_label_list

def compute_metrics(p):
    preds_list, out_label_list = align_predictions(p.predictions,p.label_ids)
    return {
        "accuracy_score": accuracy_score(out_label_list, preds_list),
        "precision": precision_score(out_label_list, preds_list),
        "recall": recall_score(out_label_list, preds_list),
        "f1": f1_score(out_label_list, preds_list),
    }

# Training

In [30]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 3e-5,
    fp16 = True, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 8, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 1, # use this to scale batch size without needing more memory
    num_train_epochs= 2,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'f1',
    greater_is_better = True,
    seed = 25
  )
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark = False

set_seed(25)

In [31]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    model_init=model_init_trained,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy Score,Precision,Recall,F1
1,0.112300,0.096251,0.969868,0.882217,0.908633,0.895230
2,0.065400,0.095438,0.972944,0.896764,0.918847,0.907671


TrainOutput(global_step=5782, training_loss=0.11870216078017666, metrics={'train_runtime': 804.6367, 'train_samples_per_second': 57.479, 'train_steps_per_second': 7.186, 'total_flos': 1.208934537984e+16, 'train_loss': 0.11870216078017666, 'epoch': 2.0})

In [32]:
trainer.evaluate()

{'eval_loss': 14.253826141357422,
 'eval_accuracy_score': 0.02716040801431873,
 'eval_precision': 0.00034859400418312805,
 'eval_recall': 0.002518539247236603,
 'eval_f1': 0.0006124219587295646,
 'eval_runtime': 43.7544,
 'eval_samples_per_second': 75.512,
 'eval_steps_per_second': 9.439}

In [16]:
trainer.evaluate()

{'eval_loss': 14.253826141357422,
 'eval_accuracy_score': 0.02716040801431873,
 'eval_precision': 0.00034859400418312805,
 'eval_recall': 0.002518539247236603,
 'eval_f1': 0.0006124219587295646,
 'eval_runtime': 45.0139,
 'eval_samples_per_second': 73.399,
 'eval_steps_per_second': 9.175}

In [24]:
trainer.model.config.label2id = label_map
trainer.model.config.id2label = inv_label_map
trainer.save_model("output_dir")
train_dataset.tokenizer.save_pretrained("output_dir")

('output_dir/tokenizer_config.json',
 'output_dir/special_tokens_map.json',
 'output_dir/vocab.txt',
 'output_dir/added_tokens.json',
 'output_dir/tokenizer.json')

In [25]:
!cp -r /content/output_dir /content/drive/MyDrive/anlp24/nerflat-arabertv02-e3

# load model

In [26]:
# prompt: load the saved model

from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir")
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir").to(device)


In [27]:
def collate_fn(batch):
    """
    Custom collate_fn function to handle batching of InputFeatures.

    Args:
    - batch: A list of InputFeatures instances.

    Returns:
    - A dictionary with keys 'input_ids', 'attention_mask', 'token_type_ids', 'labels',
      where the values are batched tensors.
    """
    # Initialize containers for the various features
    input_ids = []
    attention_masks = []
    token_type_ids = []
    labels = []


    for item in batch:
        #print(item)
        input_ids.append(torch.tensor(item['input_ids']))  # Convert to tensor
        attention_masks.append(torch.tensor(item['attention_mask']))  # Convert to tensor
        token_type_ids.append(torch.tensor(item['token_type_ids']))  # Convert to tensor
        labels.append(torch.tensor(item['labels']))

    # Convert lists to tensors and stack them
    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)
    token_type_ids = torch.stack(token_type_ids)
    labels = torch.stack(labels)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'token_type_ids': token_type_ids,
        'label': labels
    }

# Prepare DataLoader'
batch_size = 16  # Or any batch size that fits your memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False, num_workers=5)
test_loader = DataLoader(dev_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=False, num_workers=5)

KNN

In [28]:
def _normalizer(x, epsilon=1e-10) -> np.ndarray:
        return x / (np.linalg.norm(x, axis=-1, keepdims=True) + epsilon)

In [29]:
def create_index(embeddings, label_ids):
    batch_size, seq_len, _ = embeddings.shape
    label_ids = label_ids.detach().cpu().numpy()
    embeddings = embeddings.detach().cpu().numpy()

    out_label_list = []
    in_embeddings = []

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                out_label_list.append(inv_label_map[label_ids[i][j]])
                in_embeddings.append(embeddings[i][j])
    return in_embeddings, out_label_list


In [30]:
from tqdm import tqdm

In [31]:
train_embeddings = []
train_labels = []

model.eval()
for batch in tqdm(train_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['label']
        outputs = model(input_ids, attention_mask, token_type_ids,output_hidden_states=True)
        embeddings = outputs.hidden_states[-1]

        in_embeddings, in_labels = create_index(embeddings, labels)
        train_embeddings.append(in_embeddings)
        train_labels.append(in_labels)

100%|██████████| 1446/1446 [07:00<00:00,  3.44it/s]


In [32]:
len(train_embeddings), len(train_labels)

(1446, 1446)

In [33]:
datastore_keys = [item for sublist in train_embeddings for item in sublist]
datastore_labels = [item for sublist in train_labels for item in sublist]

In [34]:
len(datastore_keys), len(datastore_labels)

(390900, 390900)

In [35]:
1446*16*512

11845632

In [36]:
datastore_keys = np.array(datastore_keys)
datastore_labels = np.array(datastore_labels)

In [37]:
datastore_keys.shape, datastore_labels.shape

((390900, 768), (390900,))

In [38]:
datastore_values = [label_map[label] for label in datastore_labels]
datastore_values = np.array(datastore_values)

In [50]:
# prompt: save the numpy arrays datastore_keys and datastore_values

import numpy as np
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir/datastore_keys.npy', datastore_keys)
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/output_dir/datastore_values.npy', datastore_values)


In [40]:
def choose(embeddings, label_ids, logits):
    batch_size, seq_len, _ = embeddings.shape
    label_ids = label_ids.detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    embeddings = _normalizer(embeddings.detach().cpu().numpy())

    out_label_list = []
    in_embeddings = []
    in_logits = []

    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != torch.nn.CrossEntropyLoss().ignore_index:
                out_label_list.append(inv_label_map[label_ids[i][j]])
                in_embeddings.append(embeddings[i][j])
                in_logits.append(logits[i][j])
    return in_embeddings, out_label_list, in_logits


In [41]:
test_embeddings = []
test_labels = []
test_logits = []

model.eval()
for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['label']
        outputs = model(input_ids, attention_mask, token_type_ids,output_hidden_states=True)
        embeddings = outputs.hidden_states[-1]
        logits = outputs.logits
        in_embeddings, in_labels, in_logits = choose(embeddings, labels, logits)
        test_embeddings.append(in_embeddings)
        test_labels.append(in_labels)
        test_logits.append(in_logits)

100%|██████████| 207/207 [01:05<00:00,  3.14it/s]


In [42]:
len(test_logits[0][0])

42

In [43]:
test_embeddings = [item for sublist in test_embeddings for item in sublist]
test_labels = [item for sublist in test_labels for item in sublist]
test_logits = [item for sublist in test_logits for item in sublist]

In [44]:
test_embeddings = np.array(test_embeddings)
test_labels = np.array(test_labels)
test_logits = np.array(test_logits)

In [45]:
test_values = [label_map[label] for label in test_labels]
test_values = np.array(test_values)

In [46]:
test_embeddings.shape, test_values.shape, test_logits.shape

((57547, 768), (57547,), (57547, 42))

In [49]:
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/test_embeddings.npy', test_embeddings)
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/test_labels.npy', test_values)
np.save('/content/drive/MyDrive/anlp24/nerflat-arabertv02-e3/test_logits.npy', test_logits)


# KNN

# Conformal Prediction